In [13]:
import pandas as pd

segments = pd.read_csv("./../data/Raw-GPS-data-Kandy-Buses/MAIN/OUTPUTS/merged_segements_10T.csv")
gps_data = pd.read_csv("./../data/Raw-GPS-data-Kandy-Buses/MAIN/OUTPUTS/merged_gps_data_10T.csv")


In [15]:
gps_for_seg = gps_data[gps_data['segment_id']==1]

In [17]:
gps_arr = []
for index, row in gps_for_seg.iterrows():
    gps_arr.append((row['latitude'], row['longitude']))

In [62]:
import colorsys

def value_to_hex_color(value, max = 40.0):
    # Normalize the input value to the range [0, 1]
    normalized_value = value / max

    # Map the normalized value to the hue in the HSV color space
    hue = normalized_value * (120 / 360)  # 120 degrees for green to red

    # Set saturation and value to 1 for full color intensity
    saturation = 1.0
    brightness = 1.0

    # Convert HSV to RGB
    rgb = colorsys.hsv_to_rgb(hue, saturation, brightness)

    # Scale RGB values to the range [0, 255]
    red = int(rgb[0] * 255)
    green = int(rgb[1] * 255)
    blue = int(rgb[2] * 255)

    # Convert RGB to hexadecimal
    hex_color = "#{:02X}{:02X}{:02X}".format(red, green, blue)

    # Return the hexadecimal color code
    return hex_color

def value_to_hex_color_routes(value, max = 40.0):
    # Normalize the input value to the range [0, 1]
    normalized_value = (value + 20) / max

    # Map the normalized value to the hue in the HSV color space
    hue = normalized_value * (120 / 360)  # 120 degrees for green to red

    # Set saturation and value to 1 for full color intensity
    saturation = 1.0
    brightness = 1.0

    # Convert HSV to RGB
    rgb = colorsys.hsv_to_rgb(hue, saturation, brightness)

    # Scale RGB values to the range [0, 255]
    red = int(rgb[0] * 255)
    green = int(rgb[1] * 255)
    blue = int(rgb[2] * 255)

    # Convert RGB to hexadecimal
    hex_color = "#{:02X}{:02X}{:02X}".format(red, green, blue)

    # Return the hexadecimal color code
    return hex_color

In [65]:
import folium
# Sample GPS points (latitude, longitude)
gps_points = gps_arr

# Create a folium map centered at the mean of the GPS points
map_center = [sum(x[0] for x in gps_points) / len(gps_points),
              sum(x[1] for x in gps_points) / len(gps_points)]
mymap = folium.Map(location=map_center, zoom_start=15)

# Plot the GPS points on the map
count = 1
for index, row in gps_for_seg.iterrows():
    # folium.Marker(location=point,  icon=folium.Div).add_to(mymap)
    folium.CircleMarker(location=(row['latitude'], row['longitude']), radius=6, color='black', popup=f"{count}: {row['speed']}", fill_opacity=1.0, fill_color=value_to_hex_color(row['speed'])).add_to(mymap)
    count+=1

gps_for_seg.reset_index(drop=True)

for i in range(0, len(gps_for_seg)-1):
    locs = [(gps_for_seg.loc[i,'latitude'],gps_for_seg.loc[i,'longitude']),
            (gps_for_seg.loc[i+1,'latitude'],gps_for_seg.loc[i+1,'longitude'])]
    elev_diff = gps_for_seg.loc[i+1,'elevation']-gps_for_seg.loc[i,'elevation']
    route_line = folium.PolyLine(locations=locs, color=value_to_hex_color_routes(elev_diff,10), weight=2.5, opacity=1, tooltip=elev_diff)
    mymap.add_child(route_line)
    # print(elev_diff)
# Save the map to an HTML file or display it in a Jupyter notebook
mymap.save('gps_map.html')
